Import libraries

In [8]:
import os
import sys
import urllib2
import requests 
import StringIO
import zipfile
import numpy as np
import pandas as pd # pandas
import matplotlib.pyplot as plt # module for plotting 

from zipfile import ZipFile
from collections import defaultdict

In [10]:
# load the Lahman database
url = urllib2.urlopen('http://seanlahman.com/files/database/baseballdatabank-2017.1.zip')
zipfile = ZipFile(StringIO.StringIO(url.read()))

Read in csv files

In [38]:
# FanGraphs data
hitting = pd.read_csv('Hitting_16.csv')
fielding = pd.read_csv('Fielding_16.csv')

# Lahman data (player salaries)
salaries = pd.read_csv(zipfile.extract('baseballdatabank-2017.1/core/Salaries.csv'))
master = pd.read_csv(zipfile.extract('baseballdatabank-2017.1/core/Master.csv'))
salaries.head()

,yearID,teamID,lgID,playerID,salary
0,1985,ATL,NL,barkele01,870000
1,1985,ATL,NL,bedrost01,550000
2,1985,ATL,NL,benedbr01,545000
3,1985,ATL,NL,campri01,633333
4,1985,ATL,NL,ceronri01,625000


Filter the fielding dataset to get each player's primary position and add it as a column to the 'hitting' DataFrame

In [15]:
# remove pitchers
fielding = fielding[fielding.Pos != 'P']

# function that can be applied that will get positions
# based on number of games played there
def find_pos(df):
    positions = df.Pos
    d = defaultdict(int)
    for pos in positions:
        d[pos] += 1
    result = max(d.iteritems(), key=lambda x: x[1])
    return result[0]

positions = fielding.groupby('playerid').apply(find_pos)
positions = positions.reset_index()
positions = positions.rename(columns={0:'Pos'})
positions = positions.merge(hitting, on='playerid', how='inner')

From the Lahman database, construct a DataFrame with player salaries and full names. Then, use the full names to add player salaries to the FanGraphs data.

In [39]:
# get player salaries for the year in question
salaries = salaries[salaries.yearID == 2016]

# add full names to salaries
salaries = salaries[['playerID', 'salary']]
master = master[['playerID', 'nameLast', 'nameGiven']]
salaries = master.merge(salaries)
salaries.head()

,playerID,nameLast,nameGiven,salary
0,abadfe01,Abad,Fernando Antonio,1250000
1,abreujo02,Abreu,Jose Dariel,11666667
2,ackledu01,Ackley,Dustin Michael,3200000
3,adamecr01,Adames,Cristhian Pascual,509500
4,adamsma01,Adams,Matthew James,1650000


In [40]:
salaries

,playerID,nameLast,nameGiven,salary
0,abadfe01,Abad,Fernando Antonio,1250000
1,abreujo02,Abreu,Jose Dariel,11666667
2,ackledu01,Ackley,Dustin Michael,3200000
3,adamecr01,Adames,Cristhian Pascual,509500
4,adamsma01,Adams,Matthew James,1650000
5,adriaeh01,Adrianza,Ehire Enrique,512500
6,ahmedni01,Ahmed,Nicholas Mark,521600
7,alberha01,Alberto,Hanser Joel,511500
8,alberma01,Albers,Matthew James,2000000
9,allenco01,Allen,Cody Edward,4150000
